# Парсер отзывов с сайта banki.ru

## Подготовка к написанию парсера 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import time as t
import random
from tqdm import tqdm
import concurrent.futures

## Задача

Посмотрим, как выглядит отзыв и какие данные нам потребуется из него вынести.

Пример отзыва - https://www.banki.ru/services/responses/bank/response/11246599/

Из отзыва нам потребуется: 
* имя пользователя, оставившего отзыв - текст
* название банка, на который оставлен отзыв - текст
* общая оценка - число
* оценки прозрачности условий, вежливости сотрудников, доступности и поддержки, удобства приложения и сайта - числа
* заголовок отзыва - текст
* текст отзыва - текст
* дата отзыва - дата
* url - текст

Все проверенные отзывы можно просмотреть по ссылке - https://www.banki.ru/services/responses/list/?page=1&is_countable=on

Отзывы хранятся в объектах в виде href="/services/responses/bank/response/11254715/" 

Мы соберём ссылки на отзывы в один список.





In [2]:
# параметры для requests.get()
st_accept = 'text/html'
st_useragent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
TIMEOUT = 25
headers = {
    "Accept": st_accept,
    "User-Agent": st_useragent
}


In [3]:
session = requests.Session()
session.headers = headers

## Сбор ссылок

Код функции сбора ссылок на отзывы. (не собирает ничего больше)

In [4]:

def get_hrefs(url):
    r = requests.get(url, headers=headers, timeout=TIMEOUT)
    page_number = url[url.find('=') + 1:url.find("&")]
    bs = BeautifulSoup(r.text)
    links = []
    for a in bs.find_all("a", href=True):
        link = a['href']
        if r"/services/responses/bank/response/" in link and "#comments" not in link:
            link_code = link[link.rfind('response/') + 8::]
            if link_code not in links:
                links.append(link_code)
    return (links, page_number)

На момент 15.12.23 на сайте примерно 18900 страниц с отзывами, прошедшими модерацию

In [5]:
# занесем все страницы для сбора в список
urls = [f"https://www.banki.ru/services/responses/list/?page={page_number}&is_countable=on" for page_number in range(1, 18900)]

Далее следует код сбора ссылок
#### Внимание!
На выполнение этого блока кода уйдёт примерно 4-5 часов.

In [6]:
res = []
f = open("review_links.txt", 'a')  #файл хранящий ссылки
err = open('errors.txt', 'a')  #файл хранящий возникающие ошибки
completed_pages = open('completed_pages.txt', 'a') #файл хранящий номера отработанных страниц
err_counter = 0
CONNECTIONS = 6  #кол-во потоков
with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = (executor.submit(get_hrefs, url) for url in urls)
    for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(urls)):
        try:
            data = future.result()
            add_row = True
        except Exception as exc:
            add_row = False
            err_counter += 1
            err.write(str(exc) + "\n")
        finally:
            if add_row:
                for el in data[0]:
                    f.write(el + '\n')
                completed_pages.write(data[1] + ",")
            else:
                pass
f.close()
err.close()
completed_pages.close()
print('errors occured:', err_counter)

100%|██████████| 33444/33444 [7:57:28<00:00,  1.17it/s]   

errors occured: 1124


In [11]:
completed_pages = open("completed_pages.txt", "r")
review_links = open('review_links.txt', "r")
cleared_links = set(review_links.readlines())
comp_p = completed_pages.readline().replace("-,", '').split(",")
print(f"Спарсилось ссылок с {len(comp_p)} страниц. Всего {len(cleared_links)} ссылок на отзывы")
completed_pages.close()
review_links.close()

Спарсилось ссылок с 54455 страниц. Всего 463813 ссылок на отзывы


## Парсер

Посмотрим, как выглядит страничка с отзывом

In [80]:
url = "https://www.banki.ru/services/responses/bank/response/11273898/"
r = requests.get(url)
r

<Response [200]>

In [ ]:
bs = BeautifulSoup(r.text)
#bs

Текст отзыва

In [82]:
bs.find("div", {"class": "lb1789875"}).text

'\nБанк обманывает на кешбэк. Были приобретены билеты на сумму более 1 млн рублей. Перед приобретением был момент согласования через приложение и по телефону с сотрудниками банка возможность начисления кешбэка. начисление кешбэка подтвердили устно и письменно. Билеты приобретены были для родственника, у которого нет карты Тинькофф о чем тоже заранее сообщили банку. Кешбек был в сумме 79000 рубле...банк обманнывает уже не в первый раз. Есть возможность как то повлиять на это?\xa0\n'

заголовок отзыва, имя пользователя и название банка

In [51]:
bs.find('title')

<title>Прекрасный банк, прекрасные работники. Обратилась в банк, помогала девушка (Данилина Елена) очень добрый и отзывчивый человек. Помогла выбрать кредит и его условия.очень приятно.Спасибо большое за таких сотрудников. Обязательно приду ещё к вам в банк !!!! – отзыв о Банке Русский Стандарт от "user-290812840908" | Банки.ру</title>

дата и время отзыва

In [30]:
bs.find('span', {'class': 'l10fac986'}).text

'\n\t\t\t12.12.2023 19:53\n\t\t'

оценка отзыва

In [34]:
bs.find("div", {'class': "rating-grade"}).text

'5'

дополнительные оценки

In [49]:
additional_grades = {}
for txts6 in bs.find_all('div', {"class": 'text-size-6'}):
    additional_grades[txts6.text] = 0
for grade in bs.find_all('div', {'class': 'ld017b199'}):
    for key in additional_grades.keys():
        additional_grades[key] = str(grade).count("l61f54b7b")
print(additional_grades)

{'Прозрачные условия': 3, 'Вежливые сотрудники': 3, 'Доступность и поддержка': 3, 'Удобство приложения, сайта': 3}


Функция для сбора всего необходимого с страницы с отзывом

In [4]:
def get_review(url):
    r = requests.get(url)
    bs = BeautifulSoup(r.text)
    review_text = bs.find("div", {"class": "lb1789875"}).text
    title = str(bs.find('title'))
    review_title = title[7:title.rfind(" – отзыв о "):]
    bank_name = title[title.rfind(" – отзыв о ") + 11:title.rfind(' от "'):]
    user_name = title[title.rfind(' от "') + 5: title.rfind('"')]
    dt = bs.find('span', {"class": 'l10fac986'}).text.replace("\n", '').replace('\t', '')
    main_grade = bs.find("div", {'class': "rating-grade"}).text
    additional_grades = {}
    for txts6 in bs.find_all('div', {"class": 'text-size-6'}):
        additional_grades[txts6.text] = 0
    i = 0
    for grade in bs.find_all('div', {'class': 'ld017b199'}):
        current_key = list(additional_grades.keys())[i]
        additional_grades[current_key] = str(grade).count("l61f54b7b")
        i += 1
    clear_conditions_grade = additional_grades.get('Прозрачные условия', 0)
    staff_grade = additional_grades.get('Вежливые сотрудники', 0)
    support_grade = additional_grades.get('Доступность и поддержка', 0)
    app_site_grade = additional_grades.get('Удобство приложения, сайта', 0)
    row = [user_name, bank_name, dt, review_title, review_text, main_grade, clear_conditions_grade, staff_grade, support_grade, app_site_grade, url]
    return row

Результат работы функции

In [5]:
line = 'user_name, bank_name, dt, review_title, review_text, main_grade, clear_conditions_grade, staff_grade, support_grade, app_site_grade, url'.split(', ')
res = get_review('https://www.banki.ru/services/responses/bank/response/11014706/')
for i in range(len(res)):
    print(line[i])
    print(res[i])

user_name
user-624114719643
bank_name
Совкомбанке
dt
08.08.2023 01:00
review_title
Потеря процентов из-за ответа оператора поддержки
review_text

Дебетовая карта с ПНО. Не начислены проценты 06.08. Считают, что не выполнен оборот (не хотят учитывать покупки по СБП через QR код). Оператору поддержки, мною  был предоставлен скриншот с ответом её коллеги о том, что данная операция учитывается в обороте для моей карты. Я сообщила также, что если бы ответ по участию данной операции в обороте был бы отрицательный, то я бы , естественно, выполнила оборот другими тратами..Т.е. невыполнение оборота связано исключительно с соответствующим ответом на мой вопрос оператором поддержки... Девушка согласилась, что ситуация неприятная и перевела на Главного Специалиста...Алексей И-ев(так он был предоставлен)после повторения пару раз мне условий по выплате процентов вдруг выдал, что ответ сотрудника, об учёте данной операции в обороте верен !?!?!? И ЧТО С 1 АВГУСТА ПОМЕНЯЛИСЬ УСЛОВИЯ ПО КАРТЕ !?!?!? На 

Запрос для создания БД, в которой будут храниться отзывы

In [7]:
import sqlite3


DB_NAME = "reviews.db"
CREATE_QUERY = '''
CREATE TABLE IF NOT EXISTS reviews(
id INTEGER PRIMARY KEY,
username TEXT,
bank_name TEXT,
review_dtt TEXT,
review_title TEXT,
review_text TEXT,
main_grade INTEGER,
clear_conditions_grade INTEGER,
staff_grade INTEGER,
support_grade INTEGER,
app_site_grade INTEGER,
url TEXT
)
'''

Преобразование ссылок из файла в рабочий вид

In [9]:
review_links_f = open("review_links.txt", 'r')
review_links = review_links_f.readlines()
for i in range(len(review_links)):
    review_links[i] = int(review_links[i].replace("\n", '').replace("/", ''))
review_links = list(set(review_links))
review_links = sorted(review_links, reverse=True)
for i in range(len(review_links)):
    review_links[i] = 'https://www.banki.ru/services/responses/bank/response/' + str(review_links[i])

Подключение к БД

In [13]:
db_connection = sqlite3.connect(DB_NAME)
cursor = db_connection.cursor()
cursor.execute(CREATE_QUERY)
db_connection.commit()

Далее следует код парсера
#### Внимание!
Выполнение этого блока кода занимает много времени

In [11]:
COLUMNS = '''(username, bank_name, review_dtt, review_title, review_text, main_grade,
clear_conditions_grade, staff_grade, support_grade, app_site_grade, url)'''
ADD_QUERY = f'''
INSERT INTO reviews {COLUMNS} VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) 
'''
err_counter = 0  #счётчик ошибок
CONNECTIONS = 6  #количество потоков
err = open('errors.txt', 'a')  #файл с ошибками
with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_review = (executor.submit(get_review, url) for url in review_links)
    for future in tqdm(concurrent.futures.as_completed(future_to_review), total=len(review_links)):
        try:
            data = future.result()
            add_row = True
        except Exception as exc:
            add_row = False
            err_counter += 1
            err.write(str(type(exc)) + "\n")
        finally:
            if add_row:
                cursor.execute(ADD_QUERY, data)
                db_connection.commit()
            else:
                pass
err.close()
print('errors occured:', err_counter)

100%|██████████| 463813/463813 [50:42:18<00:00,  2.54it/s]       

errors occured: 21467


### Дозапись в БД

Во время парсинга было потеряно больше 20 тысяч отзывов, допарсим их отдельно

In [28]:
cursor.execute("""SELECT url FROM reviews""")
db_urls = cursor.fetchall()
db_urls = [url[0] for url in db_urls]
db_urls[:20]

In [36]:
not_parsed = list(set(review_links).difference(set(db_urls)))
print("Не спарсилось отзывов:", len(not_parsed))

21467


In [37]:
err_counter = 0
CONNECTIONS = 6
err = open('errors.txt', 'a')
with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_review = (executor.submit(get_review, url) for url in not_parsed)
    for future in tqdm(concurrent.futures.as_completed(future_to_review), total=len(not_parsed)):
        try:
            data = future.result()
        except Exception as exc:
            data = ['miss'] * 11
            err_counter += 1
            err.write(str(type(exc)) + "\n")
        finally:
            cursor.execute(ADD_QUERY, data)
            db_connection.commit()
err.close()
print('errors occured:', err_counter)

100%|██████████| 21467/21467 [2:47:31<00:00,  2.14it/s]      

errors occured: 2492


In [38]:
db_connection.close()  # закрываем подключение к БД